# Get more features based on open source code

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
% cd /content/drive/MyDrive/QuoraQuestionPair/data

/content/drive/MyDrive/QuoraQuestionPair/data


## oof feature

In [ ]:
%run generate_feature_oof.py

Number of right speakers on the right: 0
Number of outsiders on the left:  0


/content/drive/My Drive/QuoraQuestionPair/data/generate_feature_oof.py:226: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  total=n_splits):


TypeError: ignored

## len feature

In [ ]:
%run generate_feature_len.py

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## embedding feature

In [ ]:
from keras.utils.data_utils import get_file

In [ ]:
get_file('glove.840B.300d.zip', 'http://nlp.stanford.edu/data/glove.840B.300d.zip',cache_subdir = '/content/drive/MyDrive/QuoraQuestionPair/data/corpora')

'/content/drive/MyDrive/QuoraQuestionPair/data/corpora/glove.840B.300d.zip'

In [ ]:
get_file('wiki.en.zip', 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip', cache_subdir = '/content/drive/MyDrive/QuoraQuestionPair/data/corpora')

10356883456/10356881291 [==============================] - 722s 0us/step


'/content/drive/MyDrive/QuoraQuestionPair/data/corpora/wiki.en.zip'

In [ ]:
# unzip google new corpus
# prepare: google new corpus / fasttext / glove
%run generate_feature_embedding.py

## match feature

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
%run generate_feature_match.py

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zzb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


ZeroDivisionError: division by zero

In [ ]:
%run generate_feature_match_2.py

d:\MyCode\python\ECE449Project\QuoraQuestionPair\data\generate_feature_match_2.py:67: RuntimeWarning: invalid value encountered in true_divide
  Rcosine = np.dot(shared_weights, shared_weights)/Rcosine_denominator
d:\MyCode\python\ECE449Project\QuoraQuestionPair\data\generate_feature_match_2.py:62: RuntimeWarning: invalid value encountered in double_scalars
  R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
d:\MyCode\python\ECE449Project\QuoraQuestionPair\data\generate_feature_match_2.py:62: RuntimeWarning: invalid value encountered in long_scalars
  R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share


## question intersect feature

In [ ]:
%run generate_feature_graph_question_intersect.py

## clique feature

In [ ]:
%run generate_feature_g

#### step by step running clique feature

In [4]:
import networkx as nx
import numpy as np
import pandas as pd
from tqdm import tqdm
#import matplotlib.pyplot as plt

In [5]:
path=''
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')
y_train=train.is_duplicate.values
data=pd.concat([train[['question1','question2']],test[['question1','question2']]]).reset_index()
data.drop('index',axis=1,inplace=True)
q_list={}
dd=data.values

In [6]:
for i in tqdm(np.arange(data.values.shape[0])):
#for i in np.arange(dd.shape[0]):
    q1,q2=dd[i]
    if q_list.setdefault(q1,[i])!=[i]:
        q_list[q1].append(i)
    if q_list.setdefault(q2,[i])!=[i]:
        q_list[q2].append(i)
data['question1_link']=data.question1.map(q_list)
data['question2_link']=data.question2.map(q_list)

100%|██████████| 2750086/2750086 [00:14<00:00, 193816.15it/s]


In [7]:
data['adj_node']=data.apply(lambda x:x['question1_link']+x['question2_link'],axis=1)
G=nx.Graph()
cnt=0
for i,adj_list in tqdm(enumerate(data.adj_node.values)):
#for i,adj_list in enumerate(data.adj_node.values):
    edges=[(i,item) for item in adj_list if item !=i]
    if edges==[]:
#        if cnt%100000==0:
#            print("ignore cnts %d" %cnt)
        cnt+=1
    else:
        G.add_edges_from(edges)  
print("ignore cnts %d" %cnt)

2750086it [01:52, 24488.83it/s]

ignore cnts 2050998


In [8]:
print('begin train')
cnt=0
max_clique=np.zeros(data.shape[0])
for clique in tqdm(nx.find_cliques(G)):
#for clique in nx.enumerate_all_cliques(G):
    if cnt%100000==0:
        print("deal cnts %d" %cnt)
    len_clique=len(clique)
    for item in clique:
        max_clique[item]=max(max_clique[item],len_clique)
    cnt+=1

0it [00:00, ?it/s]

begin train


257it [00:19, 13.78s/it]

deal cnts 0


100783it [03:28, 6044.72it/s]

...

500097it [05:29, 2326.21it/s]

deal cnts 500000


599963it [39:12, 352.58it/s]

deal cnts 600000


695011it [43:35, 265.73it/s]


In [16]:
print("totally max_clique %d" %cnt)
print(type(max_clique))
max_clique.shape

totally max_clique 695011
<class 'numpy.ndarray'>


(2750086,)

In [14]:
print(train.shape)
print(test.shape)
print(train.shape[0]+test.shape[0])

(404290, 6)
(2345796, 3)
2750086


In [17]:
n_train = train.shape[0]
train_clique_data = max_clique[0:n_train]
test_clique_data = max_clique[n_train:]
print(train_clique_data.shape)
print(test_clique_data.shape)

(404290,)
(2345796,)


In [18]:
pd_train_clique = pd.DataFrame(train_clique_data,columns=['graph_clique_feat'])
pd_train_clique.to_csv('train_feature_graph_clique.csv', index=False)

pd_test_clique = pd.DataFrame(test_clique_data,columns=['graph_clique_feat'])
pd_test_clique.to_csv('test_feature_graph_clique.csv', index=False)